<a href="https://colab.research.google.com/github/kavyajeetbora/ETL_wages/blob/master/WIOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation and Import

In [1]:
# !pip install -q pyxlsb

In [2]:
%%time
# !wget https://dataverse.nl/api/access/datafile/199104 -O wiot.zip
!wget https://dataverse.nl/api/access/datafile/199097 -O exchange_rates.xlsx
!wget https://dataverse.nl/api/access/datafile/199095 -O sea.xlsx
!wget https://dataverse.nl/api/access/datafile/354108 -O NA_data.xlsx
!wget https://raw.githubusercontent.com/kavyajeetbora/ETL_wages/master/data/PPP_export.csv -O ppp.csv

--2024-07-23 15:39:19--  https://dataverse.nl/api/access/datafile/199097
Resolving dataverse.nl (dataverse.nl)... 194.171.144.89
Connecting to dataverse.nl (dataverse.nl)|194.171.144.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19308 (19K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘exchange_rates.xlsx’

exchange_rates.xlsx 100%[===================>]  18.86K  90.1KB/s    in 0.2s    

2024-07-23 15:39:20 (90.1 KB/s) - ‘exchange_rates.xlsx’ saved [19308/19308]

--2024-07-23 15:39:20--  https://dataverse.nl/api/access/datafile/199095
Resolving dataverse.nl (dataverse.nl)... 194.171.144.89
Connecting to dataverse.nl (dataverse.nl)|194.171.144.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5536437 (5.3M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘sea.xlsx’

sea.xlsx            100%[===================>]   5.28M  3.13MB/s    in 1.7s    

2024-07-23 15:39:23 (

In [3]:
import pandas as pd
from zipfile import ZipFile
from tqdm import tqdm
# from pyxlsb import open_workbook as open_xlsb

## SEA

In [31]:
%%time
sea = pd.read_excel("sea.xlsx", sheet_name='DATA')

CPU times: user 10.5 s, sys: 37.5 ms, total: 10.5 s
Wall time: 10.6 s


In [32]:
sea.head(3)

,country,variable,description,code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AUS,CAP,"Crop and animal production, hunting and relate...",A01,9739.77,13729.03,9726.30,11808.62,11163.98,12429.90,8706.66,11227.29,12556.71,13768.79,18441.27,20106.04,19659.28,18986.06,20525.22
1,AUS,CAP,Forestry and logging,A02,582.69,740.04,540.79,651.96,649.34,676.64,549.27,616.70,716.86,749.91,894.94,923.40,1001.44,1247.96,1206.44
2,AUS,CAP,Fishing and aquaculture,A03,869.31,1122.29,652.69,1024.72,1059.83,923.46,904.15,863.98,1146.37,1108.94,1116.14,1213.94,1182.11,1388.98,1475.22


In [33]:
year_range = list(range(2000,2015))

## Convert to USD $

In [34]:
exchange = pd.read_excel('exchange_rates.xlsx', sheet_name='EXR', header=3)
exchange.columns = list(exchange.columns[:2].str.lower()) + [int(c.replace("_", "")) for c in exchange.columns[2:]]
exchange.loc[exchange['acronym']=='ROM', 'acronym'] = 'ROU'
exchange = pd.melt(exchange, id_vars=['acronym'], value_vars=list(range(2000,2015)), var_name='Year')
exchange.head(3)

,acronym,Year,value
0,AUS,2000,0.582346
1,AUT,2000,0.923600
2,BEL,2000,0.923600


In [35]:
SEA = pd.melt(sea, id_vars=['country','variable','code'], value_vars= year_range, var_name='year')
SEA.head(3)

,country,variable,code,year,value
0,AUS,CAP,A01,2000,9739.77
1,AUS,CAP,A02,2000,582.69
2,AUS,CAP,A03,2000,869.31


In [36]:
SEA_USD = pd.merge(left=SEA,
         right=exchange,
         how='left',
         left_on=SEA['country']+SEA['year'].astype(str),
         right_on=exchange['acronym']+exchange['Year'].astype(str),
         suffixes=('', '_y')
)

SEA_USD['value'] = SEA_USD['value']*SEA_USD['value_y']
SEA_USD = SEA_USD.drop(['key_0', 'value_y', 'acronym', 'Year'], axis=1)

## Unpivot changes
SEA_USD = SEA_USD.pivot(columns=['year'], index=['country','variable', 'code'], values='value').reset_index()
SEA_USD.head(3)

year,country,variable,code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,AUS,CAP,A01,5671.916100,7106.640173,5290.163749,7697.850440,8223.153224,9494.504536,6559.162311,9414.767530,10739.314578,10895.298602,16956.729324,20753.655548,20363.082224,18386.613128,18528.013468
1,AUS,CAP,A02,339.327191,383.071345,294.137303,425.002293,478.290208,516.847404,413.792554,517.140569,613.105268,593.406783,822.896435,953.142714,1037.291552,1208.558159,1089.047356
2,AUS,CAP,A03,506.239201,580.937706,355.000049,667.998572,780.648522,705.379380,681.141402,724.499933,980.450138,877.508658,1026.289614,1253.041007,1224.429538,1345.125734,1331.673718


## Adjusted USD $

In [37]:
GO_PI = sea.loc[sea['variable']=='GO_PI'].copy()
GO_PI[GO_PI.columns[4:]] = GO_PI[GO_PI.columns[4:]]/100
GO_PI = pd.melt(GO_PI, id_vars=['country','variable', 'code'], value_vars=year_range, var_name='Year')
GO_PI.head(3)

,country,variable,code,Year,value
0,AUS,GO_PI,A01,2000,0.91576
1,AUS,GO_PI,A02,2000,0.91576
2,AUS,GO_PI,A03,2000,0.91576


In [38]:
SEA = SEA_USD.copy()
SEA = pd.melt(SEA, id_vars=['country','variable','code'], value_vars= year_range, var_name='year')
SEA.head(3)

,country,variable,code,year,value
0,AUS,CAP,A01,2000,5671.916100
1,AUS,CAP,A02,2000,339.327191
2,AUS,CAP,A03,2000,506.239201


In [39]:
adj_USD = pd.merge(left=SEA,
         right=GO_PI,
         how='left',
         left_on=SEA['country']+SEA['year'].astype(str)+SEA['code'],
         right_on=GO_PI['country']+GO_PI['Year'].astype(str)+GO_PI['code'],
         suffixes=('', '_y')
)
adj_USD['value'] = adj_USD['value']*adj_USD['value_y']
adj_USD = adj_USD.drop(['key_0', 'country_y','variable_y','code_y', 'Year', 'value_y'], axis=1)

## Unpivot changes
adj_USD = adj_USD.pivot(columns=['year'], index=['country','variable', 'code'], values='value').reset_index()
adj_USD.sample(4)

year,country,variable,code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
23187,KOR,VA_PI,B,0.036604,0.031689,0.038611,0.040927,0.049464,0.073287,0.073053,0.070241,0.082384,0.066412,0.086558,0.106800,0.107747,0.145941,0.213906
12655,FIN,COMP,U,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11496,ESP,VA,C26,2983.341584,3017.030289,2730.261139,3237.095607,3368.711744,3675.963693,3891.910420,4981.755097,5856.665444,5100.578188,4419.883800,4447.881821,3968.986224,3653.802049,3603.513100
35653,SWE,LAB,J59_J60,374.306494,395.777510,449.416858,528.559173,667.512505,693.054608,746.863221,952.617994,1060.562080,980.852559,1084.696456,1264.912867,1334.173635,1449.425162,1425.205271


## Industry PPP in Adjusted USD $



In [40]:
ppp1 = adj_USD.groupby(['code','variable']).sum().reset_index()
ppp1 = ppp1.drop(['country'], axis=1)
ppp1[ppp1['variable']=='GO'].sample(5)

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
100,C16,GO,2.739146e+05,2.636234e+05,2.753859e+05,3.102738e+05,3.810423e+05,4.071816e+05,4.511098e+05,5.311024e+05,5.590581e+05,5.292026e+05,5.648282e+05,6.769647e+05,7.382135e+05,8.377236e+05,8.682982e+05
500,H50,GO,1.798343e+05,1.854860e+05,1.930260e+05,2.315461e+05,2.889678e+05,3.336465e+05,3.594900e+05,4.449111e+05,5.292905e+05,4.004806e+05,4.589931e+05,5.181487e+05,5.477826e+05,5.688026e+05,6.157279e+05
116,C17,GO,4.390939e+05,4.155167e+05,4.132257e+05,4.483398e+05,5.029789e+05,5.345548e+05,5.887147e+05,6.774959e+05,7.726403e+05,7.060884e+05,7.979078e+05,9.166068e+05,9.020009e+05,9.381458e+05,9.555726e+05
452,G46,GO,2.374426e+06,2.319735e+06,2.393096e+06,2.687818e+06,3.130928e+06,3.424672e+06,3.764785e+06,4.391102e+06,5.151211e+06,4.575590e+06,5.319552e+06,6.262040e+06,6.533047e+06,6.889696e+06,7.215582e+06
836,Q,GO,1.874792e+06,1.990942e+06,2.207437e+06,2.583100e+06,2.964594e+06,3.251908e+06,3.530805e+06,4.006087e+06,4.531618e+06,4.729678e+06,5.077950e+06,5.686670e+06,5.922428e+06,6.212054e+06,6.554639e+06


## Industry Total in Current Dollars

In [41]:
ppp2 = SEA_USD.groupby(['code','variable']).sum().reset_index()
ppp2 = ppp2.drop(['country'], axis=1)
ppp2[ppp2['variable']=='GO'].sample(5)

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
788,N,GO,1.513877e+06,1.528481e+06,1.573150e+06,1.783519e+06,1.989855e+06,2.181219e+06,2.374693e+06,2.705715e+06,2.949002e+06,2.627109e+06,2.800468e+06,3.094821e+06,3.124793e+06,3.265580e+06,3.401792e+06
804,O84,GO,3.731891e+06,3.837787e+06,4.087199e+06,4.539622e+06,4.985178e+06,5.314170e+06,5.659341e+06,6.194429e+06,6.849874e+06,6.941249e+06,7.288377e+06,7.723978e+06,7.748140e+06,7.764141e+06,7.901780e+06
4,A01,GO,1.279156e+06,1.278273e+06,1.295117e+06,1.472492e+06,1.707955e+06,1.753473e+06,1.845375e+06,2.211571e+06,2.593182e+06,2.461586e+06,2.797304e+06,3.284306e+06,3.379918e+06,3.569139e+06,3.606844e+06
628,J62_J63,GO,7.442750e+05,7.712845e+05,8.014433e+05,8.870648e+05,9.855003e+05,1.060285e+06,1.142636e+06,1.299448e+06,1.464436e+06,1.418574e+06,1.529798e+06,1.711671e+06,1.757268e+06,1.822319e+06,1.895057e+06
340,C31_C32,GO,5.984357e+05,5.784153e+05,5.932970e+05,6.369131e+05,6.818768e+05,7.673565e+05,8.416349e+05,9.368506e+05,9.773720e+05,8.821606e+05,8.917020e+05,9.879983e+05,1.029797e+06,1.055289e+06,1.079242e+06


## PPP Ratio

In [42]:
ppp_ratio = ppp2.copy()
ppp_ratio.loc[:,year_range] =  ppp2.loc[:,year_range] / ppp1.loc[:,year_range]
ppp_ratio.sample(5)

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
263,C26,H_EMPE,0.742801,0.786361,0.808400,0.850183,0.871670,0.897690,0.930832,0.978567,0.993725,0.989874,1.0,1.008712,1.008313,0.999553,0.994851
179,C21,EMPE,1.249426,1.220274,1.207048,1.166307,1.144085,1.116522,1.096738,1.066564,1.000649,1.017252,1.0,0.968227,0.971703,0.967779,0.964711
584,J58,II,1.077792,1.057892,1.058117,1.041229,1.039939,1.035157,1.026851,1.029355,1.016296,1.004931,1.0,0.987253,0.969225,0.963960,0.944517
597,J59_J60,GO_PI,1.048265,1.008625,1.052225,1.011237,1.012631,1.013822,1.031833,1.038983,1.021453,1.017439,1.0,0.981035,0.957051,0.944660,0.925298
473,G47,II_PI,1.239033,1.206961,1.177260,1.150319,1.124947,1.100988,1.071782,1.056513,1.019644,1.006859,1.0,0.976522,0.958189,0.939430,0.937953


## Load PPP Exchange Rate


$V_{GDP} / PPP$

In [43]:
PPP = pd.read_csv('ppp.csv')
PPP = pd.melt(PPP, id_vars=['ISO code','Country', 'Variable code', 'Variable name'], value_vars=[str(i) for i in year_range], var_name='year')
PPP.head(3)

,ISO code,Country,Variable code,Variable name,year,value
0,ABW,Aruba,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,4519.512695
1,AGO,Angola,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,37817.11719
2,AIA,Anguilla,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,273.9180298


In [44]:
PPP[PPP['Country']=='India']

,ISO code,Country,Variable code,Variable name,year,value
78,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,2305846.25
269,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2001,2437321.25
460,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2002,2579105.25
651,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2003,2812778.5
842,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2004,3072820.25
1033,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2005,3538031.75
1224,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2006,3942002.25
1415,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2007,4394168.5
1606,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2008,4668518
1797,IND,India,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2009,4983383


In [45]:
NA_data = pd.read_excel('NA_data.xlsx', sheet_name='Data')
NA_data = NA_data[NA_data['year'].isin(year_range)][['countrycode', 'year', 'v_gdp']]
NA_data.head(3)

,countrycode,year,v_gdp
30,ABW,2000,3514.117257
31,ABW,2001,3601.926718
32,ABW,2002,3640.996725


In [46]:
PPP_exchange = pd.merge(left=PPP,
         right=NA_data,
         how='left',
         left_on=PPP['ISO code']+PPP['year'].astype(str),
         right_on=NA_data['countrycode']+NA_data['year'].astype(str),
         suffixes=('', '_y')
)
PPP_exchange['value'] = PPP_exchange['value'].str.replace(',','').astype(float)
PPP_exchange['value'] = PPP_exchange['v_gdp']/PPP_exchange['value']

PPP_exchange = PPP_exchange.drop(['key_0', 'countrycode','year_y','v_gdp'], axis=1)
PPP_exchange.dropna(inplace=True)
PPP_exchange

,ISO code,Country,Variable code,Variable name,year,value
0,ABW,Aruba,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,0.777543
1,AGO,Angola,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,3.241085
2,AIA,Anguilla,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,1.480888
3,ALB,Albania,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,31.437671
4,ARE,United Arab Emirates,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,1.087202
...,...,...,...,...,...,...
2852,VNM,Viet Nam,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,7333.596077
2853,YEM,Yemen,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,76.196263
2854,ZAF,South Africa,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,5.441281
2855,ZMB,Zambia,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2014,3.118285


## Industry PPP Exchange Rate

In [48]:
ppp_ratio

year,code,variable,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,A01,CAP,1.532604,1.445091,1.407384,1.310584,1.253553,1.325834,1.309874,1.172824,1.062271,1.076659,1.0,0.898106,0.847467,0.813265,0.811012
1,A01,COMP,1.721800,1.656864,1.630711,1.539202,1.420289,1.419253,1.394953,1.223595,1.098604,1.101071,1.0,0.901610,0.850099,0.812452,0.797377
2,A01,EMP,2.065999,1.843857,1.711472,1.568073,1.417445,1.405765,1.378881,1.209751,1.085401,1.092046,1.0,0.904305,0.851220,0.814324,0.805848
3,A01,EMPE,1.823923,1.643321,1.501851,1.319027,1.257939,1.286517,1.273543,1.161662,1.042366,1.054000,1.0,0.913763,0.849019,0.801305,0.801844
4,A01,GO,1.471860,1.415680,1.415324,1.343252,1.276877,1.290695,1.282046,1.153936,1.063763,1.089363,1.0,0.905897,0.854050,0.821130,0.808233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,U,K,1.675856,1.475231,1.324573,1.187493,1.206491,1.175710,1.178412,1.048955,1.086307,1.029092,1.0,0.970742,0.938227,0.905297,0.874806
892,U,LAB,1.675856,1.475231,1.324573,1.187493,1.206491,1.175710,1.178412,1.048955,1.086307,1.029092,1.0,0.970742,0.938227,0.905297,0.874806
893,U,VA,1.675856,1.475231,1.324573,1.187493,1.206491,1.175710,1.178412,1.048955,1.086307,1.029092,1.0,0.970742,0.938227,0.905297,0.874806
894,U,VA_PI,1.001971,1.001945,1.001548,1.000861,1.000693,1.000611,1.000415,1.000169,1.000190,1.000059,1.0,0.999931,0.999844,0.999745,0.999657


In [51]:
X1 = ppp_ratio.melt(id_vars=['code', 'variable'], value_vars=year_range)
X1.head(3)

,code,variable,year,value
0,A01,CAP,2000,1.532604
1,A01,COMP,2000,1.721800
2,A01,EMP,2000,2.065999


In [58]:
PPP_exchange.head(3)

,ISO code,Country,Variable code,Variable name,year,value
0,ABW,Aruba,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,0.777543
1,AGO,Angola,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,3.241085
2,AIA,Anguilla,cgdpo,Output-side real GDP at current PPPs (in mil. ...,2000,1.480888


In [62]:
PPP_exchange[PPP_exchange['ISO code'].isin(SEA['country'].unique())].to_excel('PPP_exchange.xlsx', index=False)

In [57]:
PPP_exchange_rate = pd.merge(left=PPP_exchange,
        right=X1,
        how='right',
        on = 'year',
        suffixes=('', '_y')
)
# PPP_exchange['value'] = PPP_exchange['value'].str.replace(',','').astype(float)
# PPP_exchange['value'] = PPP_exchange['v_gdp']/PPP_exchange['value']

# PPP_exchange = PPP_exchange.drop(['key_0', 'countrycode','year_y','v_gdp'], axis=1)
# PPP_exchange.dropna(inplace=True)
PPP_exchange_rate

,ISO code,Country,Variable code,Variable name,year,value,code,variable,value_y
0,NaN,NaN,NaN,NaN,2000,NaN,A01,CAP,1.532604
1,NaN,NaN,NaN,NaN,2000,NaN,A01,COMP,1.721800
2,NaN,NaN,NaN,NaN,2000,NaN,A01,EMP,2.065999
3,NaN,NaN,NaN,NaN,2000,NaN,A01,EMPE,1.823923
4,NaN,NaN,NaN,NaN,2000,NaN,A01,GO,1.471860
...,...,...,...,...,...,...,...,...,...
13435,NaN,NaN,NaN,NaN,2014,NaN,U,K,0.874806
13436,NaN,NaN,NaN,NaN,2014,NaN,U,LAB,0.874806
13437,NaN,NaN,NaN,NaN,2014,NaN,U,VA,0.874806
13438,NaN,NaN,NaN,NaN,2014,NaN,U,VA_PI,0.999657


## World Input Output Analysis

In [ ]:
# # loading the temp.zip and creating a zip object
# with ZipFile("wiot.zip", 'r') as zObject:
#     zObject.extractall()

In [ ]:
# %%time
# df = pd.read_excel('WIOT2000_Nov16_ROW.xlsb', engine='pyxlsb')

In [ ]:
# df.head(10)

In [ ]:
# industries = df.iloc[5:, :3].reset_index(drop=True).copy()
# industries.columns = ['code', 'industry', 'country']
# print('Total number of industries',industries.shape[0])
# industries

In [ ]:
# value = df.iloc[5:, 4:industries.shape[0]+4].copy()
# value.columns = industries['code']
# value['industry'] = industries['code']
# value['country'] = industries['country']
# value.sample(5)

In [ ]:
# isProductive = ['K64', 'K65', 'K66', 'L68', 'M69_M70', 'N', 'O84', 'T', 'U']

# value[value['industry'].isin(isProductive)].groupby('country').sum().iloc[:,:-1].sum(axis=1)